## Automatic Short Answer Scoring menggunakan Python 
Ada beberapa tahapan yang diperlukan dalam pengembangan sistem, yaitu:
1. Preparation
2. Pre-processing
3. Ekstraksi Fitur
4. Learning

### 1. Preparation
Dalam tahap preparation, kita perlu mempersiapkan reference answers dan kamus *stopwords* terlebih dahulu.
* Stopwords merupakan kata-kata yang biasanya tidak signifikan secara makna (non-content words) (i.e. yang, dari, ke, dia, kamu, etc). Kita bisa juga menambahkan dalam list *stopwords* , kata-kata yang tidak baku (yg, karna). Stopwords di list dalam file **stopwords.txt**
* Reference answer diambil berdasarkan petunjuk soal. 
Selain itu data essay (Maccu Pichu/Jaket) juga di-import.


In [33]:
import sys
import numpy as np
import csv
import pandas as pd

filename = "/home/yunita/Data/Project/essay_scoring/Dataset/macu_picu/maccu-piccu_rescore.xlsx"
df = pd.read_excel(filename, sheetname='Sheet1', header=0)
df_ = df[df['rescore_ori'] != 9]  # membuang row yang kosong
ans1 = np.array(df_['fte_data1'])
ans2 = np.array(df_['fte_data2'])
label = np.array(df_['rescore_ori'])

print ans1[0]

print ("loading data")
print filename
print len(ans1)
print len(ans2)
print set(label)

Biro Perjalanan Buana Wisata menawarkan berbagai perjalanan wisata yang akan membantu mengubah kota batu yang sunyi ini menjadi hidup.Biro perjalanan buana wisata menawarkan akses ketempat - tempat yang jarang diketahui wisatawan.
loading data
/home/yunita/Data/Project/essay_scoring/Dataset/macu_picu/maccu-piccu_rescore.xlsx
6618
6618
set([0, 1])


Memanggil kamus *stopwords* yang terdiri dari 765 kata. 

In [34]:
fstopw = open('/home/yunita/Data/Code/essay_scoring/stopwords.txt','r')
stopw = set([w for w in fstopw])
stopadd = ['karena','dia','adalah','jadi','yg','dan','ia','ini']
for s in stopadd:
  stopw.add(s)
print (len(stopw))

765


### 2. Pre-processing
Setelah data di-import dan kamus stopword sudah dipersiapkan, maka tahap berikutnya adalah melakukan pre-processing dengan membuang semua *stopwords* dan menyeragamkan jawaban dalam huruf kecil (*lowercasing*), untuk meminimalisir perbedaan penulisan jawaban.


In [38]:
answer = []
for i in range(len(label)):
    ans = []
    if ans1[i] is np.nan:
        ans = ans2[i]
    elif ans2[i] is np.nan:
        ans = ans1[i]
    else:
        ans = ans1[i]+ans2[i]
    ans_lower = ans.lower()
    ans_stop = [word for word in ans_lower.split() if word not in stopw]
    answer.append(' '.join(ans_stop))

### 3. Ekstraksi Fitur
Tahap berikutnya adalah ekstraksi fitur. Untuk eksperimen kali ini, kita akan menggunakan frekuensi kata yang terdapat pada jawaban sebagai fitur. Untuk ekstraksi fitur, digunakan salah satu python library yang disebut Scikit Learn

In [36]:
from sklearn.feature_extraction.text import CountVectorizer
count_vect = CountVectorizer(analyzer="word")
train_data = count_vect.fit_transform(answer)

### 4. Learning
Tahap selanjutnya adalah learning dengan mengimplementasikan algoritma machine learning. Kita bisa mencoba beberapa algoritma dengan *cross-validation* untuk melihat algoritma mana yang paling optimal. Pada experiment dengan menggunakan 5-cross validation, dengan 4 metode yang berbeda, diperoleh akurasi tertinggi yaitu 97.26% dengan menggunakan Ada Boost Classifier

In [40]:
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.naive_bayes import MultinomialNB
from sklearn.model_selection import cross_val_score
from sklearn import svm
from sklearn.ensemble import RandomForestClassifier, AdaBoostClassifier

clf1 = MultinomialNB()
clf2 = svm.SVC(probability=True, random_state=0)
clf3 = RandomForestClassifier(max_depth=5, n_estimators=10, max_features=1)
clf4 = AdaBoostClassifier()

print ("Naive Bayes:", np.mean(cross_val_score(clf1, train_data, label, scoring='accuracy',cv=5))) 
print ("SVM:", np.mean(cross_val_score(clf2, train_data, label, scoring='accuracy',cv=5))) 
print ("Random Forest:", np.mean(cross_val_score(clf3, train_data, label, scoring='accuracy',cv=5))) 
print ("Ada Boost:", np.mean(cross_val_score(clf4, train_data, label, scoring='accuracy',cv=5))) 

('Naive Bayes:', 0.9468109986867814)
('SVM:', 0.9530045832744669)
('Random Forest:', 0.6199748863703392)
('Ada Boost:', 0.972648557616058)
